# 1. Ad.Group

In [67]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 6
import os
os.chdir('/Users/JayLiang/Desktop/Media Storm/FFC/DoubleClick Search')
import argparse
import httplib2
import pprint
import time
import datetime
from io import StringIO
from pandas import ExcelWriter

from googleapiclient.discovery import build
from oauth2client import GOOGLE_TOKEN_URI
from oauth2client.client import OAuth2Credentials
from googleapiclient.errors import HttpError


def create_credentials():
    """Create Google OAuth2 credentials.

    Args:
        client_id: Client id of a Google Cloud console project.
        client_secret: Client secret of a Google Cloud console project.
        refresh_token: A refresh token authorizing the Google Cloud console project
          to access the DS data of some Google user.
    Returns:
        OAuth2Credentials
    """
    return OAuth2Credentials(access_token=None,
                           client_id='549790627766-qnth4m8qvuimg87pnsp4b82lhte7dk5a.apps.googleusercontent.com',
                           client_secret='Vta4lQLOL49vVYvktkcPGRNb',
                           refresh_token='1/ab7pCGMu3K5AveG0UOUpQ0J08vCp6uM357O8qmoPDMs',
                           token_expiry=None,
                           token_uri="https://accounts.google.com/o/oauth2/token",
                           user_agent=None)

def get_service(credentials):
    """Set up a new DoubleClick Search service.

    Args:
        credentials: An OAuth2Credentials generated with create_credentials, or
        flows in the oatuh2client.client package.
    Returns:
        An authorized Doubleclicksearch serivce.
    """
    # Use the authorize() function of OAuth2Credentials to apply necessary credential
    # headers to all requests.
    http = credentials.authorize(http = httplib2.Http())

    # Construct the service object for the interacting with the DoubleClick Search API.
    service = build('doubleclicksearch', 'v2', http=http)
    return service

def poll_report(service, report_id):
    """Poll the API with the reportId until the report is ready, up to ten times.

    Args:
        service: An authorized Doublelcicksearch service.
        report_id: The ID DS has assigned to a report.
    Returns:
        pd.DataFrame, report file
    """
    for _ in range(10):
        try:
            request = service.reports().get(reportId=report_id)
            json_data = request.execute()
            if json_data['isReportReady']:
                pprint.pprint('The report is ready.')

                # For large reports, DS automatically fragments the report into multiple
                # files. The 'files' property in the JSON object that DS returns contains
                # the list of URLs for file fragment. To download a report, DS needs to
                # know the report ID and the index of a file fragment.
                report = pd.DataFrame()
                for i in range(len(json_data['files'])):
                    pprint.pprint('Downloading fragment ' + str(i) + ' for report ' + report_id)
                    report = report.append(download_files(service, report_id, str(i)), ignore_index = True) # See Download the report.
                return report

            else:
                pprint.pprint('Report is not ready. I will try again.')
                time.sleep(10)
        except HttpError as e:
            error = simplejson.loads(e.content)['error']['errors'][0]

            # See Response Codes
            pprint.pprint('HTTP code %d, reason %s' % (e.resp.status, error['reason']))
            break
        
def download_files(service, report_id, report_fragment):
    """Generate and print sample report.

    Args:
        service: An authorized Doublelcicksearch service.
        report_id: The ID DS has assigned to a report.
        report_fragment: The 0-based index of the file fragment from the files array.
    Returns:
        pd.DataFrame report file
    """
    request = service.reports().getFile(reportId=report_id, reportFragment=report_fragment)
    return pd.read_csv(StringIO(request.execute().decode('utf-8')))



In [6]:
def request_report_adgroup(service, start_date, end_date, columns):
    """Request sample report and print the report ID that DS returns. See Set Up Your Application.

    Args:
        service: An authorized Doublelcicksearch service.
        columns: list of columns will be in the report
    Returns:
        The report id.
    """
    request = service.reports().request(
        body={
                "reportScope": {
                    "agencyId": "20100000000000932",
                    "advertiserId": "21700000001365301", # 
                },
                "reportType": "adGroup",
                "columns": [{'columnName': column} for column in columns],   
                "timeRange" : {
                    "startDate" : start_date,
                    "endDate" : end_date
                    },
                
                #"filters": [
                #    {
                #        "column" : { "columnName": "keywordLabels" },
                #        "operator" : "containsElement",
                #        "values" : ["JubaNovTest",]
                #    }
                #],
                
                "downloadFormat": "csv",
                "maxRowsPerFile": 100000000,
                "statisticsCurrency": "agency",
                "verifySingleTimeZone": "false",
                "includeRemovedEntities": "false"
            }
    )
    json_data = request.execute()
    return json_data['id']


In [68]:
# Ad.group
creds = create_credentials()

service = get_service(creds)

end_date = "2017-08-14"
start_date = "2017-07-16"

print(start_date), print(end_date)
REPORTID_AdGroup_nonHVA = request_report_adgroup(service, start_date, end_date, 
                                 ['adGroup', 'impr','clicks', 'cost', 'dfaRevenue', 'dfaTransactions',
                                  'dfaActions'])
REPORTID_AdGroup_HVA = request_report_adgroup(service, start_date, end_date, 
                              ['adGroup',
                               'floodlightActivity', 'dfaActions'])

2017-07-16
2017-08-14


In [71]:
non_hva_adgroup= poll_report(service, REPORTID_AdGroup_nonHVA)
hva_adgroup = poll_report(service, REPORTID_AdGroup_HVA)

'The report is ready.'
'Downloading fragment 0 for report AAAnqvkg1bvTbYc2'
'The report is ready.'
'Downloading fragment 0 for report AAAn3qclmay8Fzrd'


In [93]:
non_hva_adgroup.rename(columns={'dfaRevenue':'revenue','dfaTransactions':'transactions',
                                  'dfaActions':'actions'}, inplace=True)
non_hva_adgroup_Agg=non_hva_adgroup.groupby(['adGroup']).sum()
non_hva_adgroup_Agg.reset_index(inplace=True)
non_hva_adgroup_Agg['ROI']=non_hva_adgroup_Agg['revenue']/non_hva_adgroup_Agg['cost']
non_hva_adgroup_Agg['ctr']=non_hva_adgroup_Agg['clicks']/non_hva_adgroup_Agg['impr']
non_hva_adgroup_Agg['cpc']=non_hva_adgroup_Agg['cost']/non_hva_adgroup_Agg['clicks']
non_hva_adgroup_Agg['cpm']=non_hva_adgroup_Agg['cost']/non_hva_adgroup_Agg['impr']*1000.0
non_hva_adgroup_Agg['cost/trans']=non_hva_adgroup_Agg['cost']/non_hva_adgroup_Agg['transactions']
non_hva_adgroup_Agg['cost/actions']=non_hva_adgroup_Agg['cost']/non_hva_adgroup_Agg['actions']

hva_adgroup=hva_adgroup.groupby(['adGroup','floodlightActivity']).sum()
hva_adgroup_wide=hva_adgroup.pivot_table(index=['adGroup'],columns=['floodlightActivity'],values=['dfaActions'])
hva_adgroup_wide.columns=hva_adgroup_wide.columns.get_level_values(1)
hva_adgroup_wide.reset_index(inplace=True)

adgroup_df=pd.merge(non_hva_adgroup_Agg,hva_adgroup_wide,how='outer',on="adGroup")
adgroup_sort=adgroup_df.sort_values(['ROI','ctr'],ascending=[0, 1])

In [95]:
adgroup_sort.to_excel("adgroup_sort.xlsx",index=False)

In [168]:
def combine(df1,df2):
    df1.rename(columns={'dfaRevenue':'revenue','dfaTransactions':'transactions',
                                  'dfaActions':'actions'}, inplace=True)
    Agg=df1.groupby(list(df1.columns[:-6])).sum()
    Agg.reset_index(inplace=True)
    Agg['ROI']=Agg['revenue']/Agg['cost']
    Agg['ctr']=Agg['clicks']/Agg['impr']
    Agg['cpc']=Agg['cost']/Agg['clicks']
    Agg['cpm']=Agg['cost']/Agg['impr']*1000.0
    Agg['cost/trans']=Agg['cost']/Agg['transactions']
    Agg['cost/actions']=Agg['cost']/Agg['actions']
    
    Agg2=df2.groupby(list(df2.columns[:-1])).sum()
    Agg2_wide=Agg2.pivot_table(index=list(df2.columns[:-2]),columns=['floodlightActivity'],values=['dfaActions'])
    Agg2_wide.columns=Agg2_wide.columns.get_level_values(1)
    Agg2_wide.reset_index(inplace=True)
    
    df=pd.merge(Agg,Agg2_wide,how='outer',on=list(Agg.columns[:-12]))
    df_sort=df.sort_values(['ROI','ctr'],ascending=[0, 1])
    return df_sort

# 2. Keyword

In [137]:
def request_report_keyword(service, start_date, end_date, columns):
    """Request sample report and print the report ID that DS returns. See Set Up Your Application.

    Args:
        service: An authorized Doublelcicksearch service.
        columns: list of columns will be in the report
    Returns:
        The report id.
    """
    request = service.reports().request(
        body={
                "reportScope": {
                    "agencyId": "20100000000000932",
                    "advertiserId": "21700000001365301", # 
                },
                "reportType": "keyword",
                "columns": [{'columnName': column} for column in columns],   
                "timeRange" : {
                    "startDate" : start_date,
                    "endDate" : end_date
                    },
                
                #"filters": [
                #    {
                #        "column" : { "columnName": "keywordLabels" },
                #        "operator" : "containsElement",
                #        "values" : ["JubaNovTest",]
                #    }
                #],
                
                "downloadFormat": "csv",
                "maxRowsPerFile": 100000000,
                "statisticsCurrency": "agency",
                "verifySingleTimeZone": "false",
                "includeRemovedEntities": "false"
            }
    )
    json_data = request.execute()
    return json_data['id']


In [138]:
# Keyword
creds = create_credentials()

service = get_service(creds)

end_date = "2017-08-14"
start_date = "2017-07-16"

print(start_date), print(end_date)
REPORTID_Keyword_nonHVA = request_report_keyword(service, start_date, end_date, 
                                 ['keywordText','campaign','adGroup','keywordMatchType','engineStatus','account',
                                  'impr','clicks', 'cost', 'dfaRevenue', 'dfaTransactions','dfaActions'])
REPORTID_Keyword_HVA = request_report_keyword(service, start_date, end_date, 
                              ['keywordText','campaign','adGroup','keywordMatchType','engineStatus','account',
                               'floodlightActivity', 'dfaActions'])


2017-07-16
2017-08-14


In [139]:
non_hva_keyword= poll_report(service, REPORTID_Keyword_nonHVA )
hva_keyword= poll_report(service, REPORTID_Keyword_HVA )

'Report is not ready. I will try again.'
'The report is ready.'
'Downloading fragment 0 for report AAAnNn9xqtcpJci3'
'The report is ready.'
'Downloading fragment 0 for report AAAnqPBoGutJtfuL'


In [177]:
keyword_sort=combine(non_hva_keyword,hva_keyword)

# 3. Date

In [173]:
def request_report_account(service, start_date, end_date, columns):
    """Request sample report and print the report ID that DS returns. See Set Up Your Application.

    Args:
        service: An authorized Doublelcicksearch service.
        columns: list of columns will be in the report
    Returns:
        The report id.
    """
    request = service.reports().request(
        body={
                "reportScope": {
                    "agencyId": "20100000000000932",
                    "advertiserId": "21700000001365301", # 
                },
                "reportType": "account",
                "columns": [{'columnName': column} for column in columns],   
                "timeRange" : {
                    "startDate" : start_date,
                    "endDate" : end_date
                    },
                
                #"filters": [
                #    {
                #        "column" : { "columnName": "keywordLabels" },
                #        "operator" : "containsElement",
                #        "values" : ["JubaNovTest",]
                #    }
                #],
                
                "downloadFormat": "csv",
                "maxRowsPerFile": 100000000,
                "statisticsCurrency": "agency",
                "verifySingleTimeZone": "false",
                "includeRemovedEntities": "false"
            }
    )
    json_data = request.execute()
    return json_data['id']


In [175]:
# Date
creds = create_credentials()

service = get_service(creds)

end_date = "2017-08-14"
start_date = "2017-07-16"

print(start_date), print(end_date)
REPORTID_Date_nonHVA = request_report_account(service, start_date, end_date, 
                                 ['date',
                                  'impr','clicks', 'cost', 'dfaRevenue', 'dfaTransactions','dfaActions'])
REPORTID_Date_HVA = request_report_account(service, start_date, end_date, 
                              ['date',
                               'floodlightActivity', 'dfaActions'])

2017-07-16
2017-08-14


In [176]:
non_hva_date= poll_report(service, REPORTID_Date_nonHVA )
hva_date= poll_report(service, REPORTID_Date_HVA )

'Report is not ready. I will try again.'
'The report is ready.'
'Downloading fragment 0 for report AAAnMev7_q0KqIYn'
'The report is ready.'
'Downloading fragment 0 for report AAAnTTftEuc8OChI'


In [178]:
date_sort=combine(non_hva_date,hva_date)

# 4. Campaign

# 5. Device

# 6. DMA

In [ ]:
7. Gender